# Joining and Appending DataFrames in PySpark HW

Now it's time to test your knowledge and further engrain the concepts we touched on in the lectures. Let's go ahead and get started.




**As always let's start our Spark instance.**

In [73]:
import findspark
findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("joins").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

You are working with 1 core(s)


## Read in the database

Let cotinue working with our college courses dataframe to get some more insights and practice what we have learned!Let's read in the whole database using the loop function that we learned about in the lecture to automatically read in all the datasets from the uw-madision-courses folder (there are too many datasets to each one individually.

In [74]:
import os

path = "C:/Users/datnh/Documents/Data Analysis/7th week/uw-madison-courses/"

df_list = []
dataframes = []
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        filename_list = filename.split(".") #separate path from .csv
        df_name = filename_list[0]
        df = spark.read.csv(path+filename,inferSchema=True,header=True)
        df.name = df_name
        df_list.append(df_name)
        dataframes.append(df)
        exec(df_name + ' = df')
        
# QA
print("Full list of dfs:")
print(df_list)

Full list of dfs:
['courses', 'course_offerings', 'grade_distributions', 'instructors', 'rooms', 'schedules', 'sections', 'subjects', 'subject_memberships', 'teachings']


Now check the contents of a few of the dataframses that were read in above.

In [55]:
dataframes[1].toPandas()

,uuid,course_uuid,term_code,name
0,344b3ebe-da7e-314c-83ed-9425269695fd,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1092,Cooperative Education Prog
1,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1082,Cooperative Education Prog
2,ea3b717c-d66b-30dc-8b37-964d9688295f,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1172,Cooperative Education Prog
3,075da420-5f49-3dd0-93df-13e3c152e1b1,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1114,Cooperative Education Prog
4,2b4e216d-a728-3713-8c7c-19afffc6b2fd,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1104,Cooperative Education Prog
...,...,...,...,...
81447,3573e335-e7be-33a6-b023-8704b3acc93b,4a7f3d9d-f2e5-3df2-8dee-2c2d02ed9098,1182,Conservation Planning
81448,5d437083-a1d0-38b8-aa70-ec3564da39a6,1220244c-2bbf-3169-8eb0-6a2da36f40f6,1182,null
81449,13278922-0726-3823-abda-dc14df03d8a6,516f637c-a244-3df0-ba3f-f11ce650543d,1182,Patient Ed. as Advocacy Strat.
81450,fd039fe4-32ec-31dd-a142-c7db54df784b,fbe0838f-9469-3d83-ad85-91ba5aabd1a3,1182,Multid. Appr. to Colon Cancer


## Recap: About this database

You will notice that there are several more tables in the uw-madision-courses folder than there are read in above. This so that you will have a chance to practice your own custom joins and learn about the relationships between a real database work. Sometimes we don't know how they are related and we need to figure it out! I'll save that for the HW :) 

Here is a look at some of the important variables we can use to join our tables:

 - course_offerings: uuid, course_uuid, term_code, name
 - instructors: id, name
 - schedules: uuid
 - sections: uuid, course_offering_uuid,room_uuid, schedule_uuid
 - teachings: instructor_id, section_uuid
 - courses: uuid
 - grade_distributions: course_offering_uuid,section_number
 - rooms: uuid, facility_code, room_code
 - subjects: code
 - subject_memberships: subject_code, course_offering_uuid
 
 **Source:** https://www.kaggle.com/Madgrades/uw-madison-courses
 
So alright, let's use this information to discover some insights from this data!

## 1a. Can you assign the room numbers to each section of each course?

Show only the rooms uuid, facility code, room number, term code and the name of the course from the course_offerings table.

In [85]:
rooms = dataframes[4]
sections = dataframes[6]
course_offerings = dataframes[1]


sections_with_rooms = sections.join(rooms, sections.room_uuid == rooms.uuid)
sections_with_rooms_and_courses = sections_with_rooms.join(course_offerings, sections_with_rooms.course_offering_uuid == course_offerings.uuid)
# sections_with_rooms_and_courses.toPandas()


result = sections_with_rooms_and_courses.select("room_uuid", "facility_code", "number", "term_code", "name")


In [81]:
result.show()

+--------------------+-------------+------+---------+--------------------+
|           room_uuid|facility_code|number|term_code|                name|
+--------------------+-------------+------+---------+--------------------+
|04368a56-c959-3e4...|   OFF CAMPUS|     1|     1172|Cooperative Educa...|
|04368a56-c959-3e4...|   OFF CAMPUS|     1|     1172|Cooperative Educa...|
|04368a56-c959-3e4...|   OFF CAMPUS|     1|     1114|Cooperative Educa...|
|04368a56-c959-3e4...|   OFF CAMPUS|     1|     1104|Cooperative Educa...|
|04368a56-c959-3e4...|   OFF CAMPUS|     1|     1134|Cooperative Educa...|
|04368a56-c959-3e4...|   OFF CAMPUS|     1|     1134|Cooperative Educa...|
|04368a56-c959-3e4...|   OFF CAMPUS|     1|     1084|Cooperative Educa...|
|04368a56-c959-3e4...|   OFF CAMPUS|     1|     1154|Cooperative Educa...|
|04368a56-c959-3e4...|   OFF CAMPUS|     1|     1154|Cooperative Educa...|
|04368a56-c959-3e4...|   OFF CAMPUS|     1|     1162|Cooperative Educa...|
|04368a56-c959-3e4...|   

## 1b. Now show same output as above but for only facility number 0469 (facility_code)

In [82]:
result_filtered = result.filter(result.facility_code == '0469')
result_filtered.show()


+--------------------+-------------+------+---------+--------------------+
|           room_uuid|facility_code|number|term_code|                name|
+--------------------+-------------+------+---------+--------------------+
|9759cb5f-a7d3-3d0...|         0469|     1|     1152|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     1|     1092|  Fundamentals-Flute|
|6af80b0b-b3e3-370...|         0469|     1|     1172|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     1|     1162|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     1|     1132|  Fundamentals-Flute|
|50322d30-dd8f-3c6...|         0469|     1|     1072|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     1|     1142|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     1|     1112|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     1|     1102|  Fundamentals-Flute|
|50322d30-dd8f-3c6...|         0469|     1|     1082|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|   

## 2. Count how many sections are offered for each subject for each facility

*Note: this will involve a groupby*


In [86]:
subjects = dataframes[8]
result2 = sections_with_rooms_and_courses.join(subjects, subjects.course_offering_uuid == sections_with_rooms_and_courses.course_offering_uuid)
result2.groupBy("facility_code", "subject_code").count().show()


+-------------+------------+-----+
|facility_code|subject_code|count|
+-------------+------------+-----+
|         0400|         270|  722|
|         0054|         900|   25|
|         0084|         120|    3|
|         0056|         184|   94|
|         0055|         424|    6|
|         0560|         900|   10|
|         0048|         424|   20|
|         0046|         888|   53|
|         0050|         754|   59|
|         0402|         778|  213|
|        0451C|         502|    2|
|         1243|         352|    4|
|         0140|         778|  125|
|         0047|         736|   19|
|         0050|         963|   21|
|         0557|         502|    5|
|         0508|         252|  648|
|         0054|         208| 1760|
|         0056|         208|   26|
|         0469|         736| 1296|
+-------------+------------+-----+
only showing top 20 rows



## 3. What are the hardest classes?

Let's see if we can figure out which classes are the hardest by seeing how many students failed. Note that you will first need to aggregate the grades table by the course uuid to include all sections. Show the name of the course as well that you will need to get from the course_offering table.

In [89]:
grades_distro = dataframes[6]

In [96]:
result3 = grades_distro.join(course_offerings, grades_distro.course_offering_uuid == course_offerings.uuid)
result3_pd = result3.toPandas()
max_value = result3_pd['number'].max()

result_filtered_2 = result3_pd[result3_pd['number'] == max_value]

In [102]:
print(result_filtered_2.name.iloc[0])

Chemical Principles I


## Challenge Question: Automating data entry errors

We see in the dataframe below that there are several typos of various animal names. If this was a large database of several millions of records, correcting these errors would be way too labor intensive. How can we automate correcting these errors?

*Hint: Leven...*

In [42]:
values = [('Monkey',10),('Monkay',36),('Mnky',123), \
          ('Elephant',48),('Elefant',16),('Ellafant',1), \
          ('Hippopotamus',48),('Hipopotamus',16),('Hippo',1)]
zoo = spark.createDataFrame(values,['Animal','age'])
zoo.show()

+------------+---+
|      Animal|age|
+------------+---+
|      Monkey| 10|
|      Monkay| 36|
|        Mnky|123|
|    Elephant| 48|
|     Elefant| 16|
|    Ellafant|  1|
|Hippopotamus| 48|
| Hipopotamus| 16|
|       Hippo|  1|
+------------+---+



In [45]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, levenshtein, when

# Create SparkSession
spark = SparkSession.builder \
    .appName("Typos Correction") \
    .getOrCreate()

# Original data
values = [('Monkey', 10), ('Monkay', 36), ('Mnky', 123),
          ('Elephant', 48), ('Elefant', 16), ('Ellafant', 1),
          ('Hippopotamus', 48), ('Hipopotamus', 16), ('Hippo', 1)]

# Create DataFrame
zoo = spark.createDataFrame(values, ['Animal', 'age'])

# Define correct spellings
corrections = {
    'Monkay': 'Monkey',
    'Mnky': 'Monkey',
    'Elefant': 'Elephant',
    'Ellafant': 'Elephant',
    'Hipopotamus': 'Hippopotamus'
}

# Create DataFrame for corrections
corrections_df = spark.createDataFrame(list(corrections.items()), ['typo', 'correction'])

# Join original DataFrame with corrections based on Levenshtein distance
zoo_corrected = zoo.join(corrections_df, levenshtein(zoo.Animal, corrections_df.typo) < 2, 'left_outer') \
                   .withColumn('Corrected_Animal',when(col('correction').isNull(), col('Animal')).otherwise(col('correction'))) \
                   .drop('typo', 'correction')

# Show corrected DataFrame
zoo_corrected.show()


+------------+---+----------------+
|      Animal|age|Corrected_Animal|
+------------+---+----------------+
|      Monkey| 10|          Monkey|
|      Monkay| 36|          Monkey|
|        Mnky|123|          Monkey|
|    Elephant| 48|        Elephant|
|     Elefant| 16|        Elephant|
|    Ellafant|  1|        Elephant|
|Hippopotamus| 48|    Hippopotamus|
| Hipopotamus| 16|    Hippopotamus|
|       Hippo|  1|           Hippo|
+------------+---+----------------+



### Great job! 